# Data Analysis for Chinook Music Group

### Introduction

Chinook Music Group is a fictional digital music shop - kind of like a mini-iTunes store. The tasks are to examine genre popularity, sales agent performance, and sales by country. Lastly, look at the impact of deciding to remove complete album purchases from the library and replacing them with only a selection of songs from each album.

### Genre Popularity in US Market

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql

--join relevant tables, group by genre, and determine market share

WITH
usa_invoice_line AS
    (
     SELECT
        il.invoice_line_id invoice_line_id,
        il.track_id track_id,
        c.country country
     FROM invoice_line il
     INNER JOIN invoice inv ON inv.invoice_id = il.invoice_id
     INNER JOIN customer c on c.customer_id = inv.customer_id
     WHERE c.country = "USA"   
    )
SELECT
    g.name genre,
    COUNT(uil.invoice_line_id) count,
    ROUND(CAST(COUNT(uil.invoice_line_id)*100 AS Float)/(SELECT COUNT(invoice_line_id) FROM usa_invoice_line), 1) percent
FROM usa_invoice_line uil
INNER JOIN track t ON t.track_id = uil.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY genre
ORDER BY count DESC;

 * sqlite:///chinook.db
Done.


genre,count,percent
Rock,561,53.4
Alternative & Punk,130,12.4
Metal,124,11.8
R&B/Soul,53,5.0
Blues,36,3.4
Alternative,35,3.3
Pop,22,2.1
Latin,22,2.1
Hip Hop/Rap,20,1.9
Jazz,14,1.3


**Genre Popularity in US Market Conclusions**
<br>
* The US market for Chinook Music Group is more than 50% comprised of tracks in the Rock genre. The second most popular genre is Alternative & Punk.
* Looking at the four options, it's safe to assume that Red Tone is a great addition.
* Blues is the next most popular genre of the three bands left, putting Slim Jim Bites as the second best addition.
* Both Pop and Hip-Hop genres have pretty similar market shares, so there may be a better reason to distinguish between Regal and Meteor and the Girls.

### Sales Support Agent Performance

In [3]:
%%sql

--create comparison table for sales team

WITH
cust_totals AS --this is the total spent for each customer
    (
     SELECT
        i.customer_id customer_id,
        CAST(SUM(i.total) AS Int) cust_total
     FROM invoice i GROUP BY 1
    ),
avg_sale AS
    (
     SELECT
        e.employee_id,
        ROUND(AVG(i.total), 2) average_sale
     FROM employee e
     INNER JOIN customer c ON c.support_rep_id = e.employee_id
     INNER JOIN invoice i ON i.customer_id = c.customer_id
     GROUP BY 1
    )
SELECT
    e.first_name || " " || e.last_name "Sales Support Agent",
    SUM(ct.cust_total) "Total Sales",
    ROUND(SUM(ct.cust_total)*100/(SELECT CAST(SUM(total) AS Float) FROM invoice), 1) "Percent Total Sales",
    avs.average_sale "Average Sale",
    COUNT(c.customer_id) "Number of Customers",
    COUNT(DISTINCT c.country) "Number of Countries"
FROM customer c
LEFT JOIN cust_totals ct ON ct.customer_id = c.customer_id
LEFT JOIN employee e ON e.employee_id = c.support_rep_id
LEFT JOIN avg_sale avs ON avs.employee_id = e.employee_id
GROUP BY 1;

 * sqlite:///chinook.db
Done.


Sales Support Agent,Total Sales,Percent Total Sales,Average Sale,Number of Customers,Number of Countries
Jane Peacock,1723,36.6,8.17,21,10
Margaret Park,1577,33.5,7.4,20,12
Steve Johnson,1389,29.5,7.41,18,13


In [4]:
%%sql

--agent sales by country

WITH cust_totals AS
    (
     SELECT
        i.customer_id customer_id,
        CAST(SUM(i.total) AS Int) cust_total
     FROM invoice i GROUP BY 1
    )
SELECT
    e.first_name || " " || e.last_name "Sales Support Agent",
    c.country "Country",
    SUM(cust_totals.cust_total) "Total Sales"
FROM customer c
LEFT JOIN cust_totals ON cust_totals.customer_id = c.customer_id
LEFT JOIN employee e ON e.employee_id = c.support_rep_id
GROUP BY 2, 1
ORDER BY 2, 3 DESC;

 * sqlite:///chinook.db
Done.


Sales Support Agent,Country,Total Sales
Margaret Park,Argentina,39
Margaret Park,Australia,81
Steve Johnson,Austria,69
Margaret Park,Belgium,60
Jane Peacock,Brazil,190
Margaret Park,Brazil,166
Steve Johnson,Brazil,69
Jane Peacock,Canada,371
Steve Johnson,Canada,91
Margaret Park,Canada,70


**Sales Support Agent Performance Conclusion**
* The three sales support agents are separated by about 3% of Chinook Music Groups total sales for 1st, 2nd, and 3rd place. Jane Peacock has the most sales, but also has the largest number of customers and the highest average sale. 
* Steve Johnson, 3rd place, has the same average sale as Margaret Park in 2nd place, but she has more customers.
* Steve has the most countries in his sales profile, so that doesn't appear to be strongly correlated to having a high percent of total sales.
* Also, when comparing sales within the same country, Jane often has the highest total but not always. Margaret and Steve do not show any particular advantage. Margaret's US market has the most sales for any country.

### Sales Data by Country

In [5]:
%%sql

--sales breakdown by country

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       i.*
     FROM invoice i
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers, -- number of customers in each country
        ROUND(SUM(total), 2) total_sales, -- total sales for the country
        ROUND(SUM(total) / count(distinct customer_id), 2) customer_lifetime_value, --average total sales per customer
        ROUND(SUM(total) / count(distinct invoice_id), 2) average_order, --average customer order
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, average_order DESC
    );

 * sqlite:///chinook.db
Done.


country,customers,total_sales,average_order,customer_lifetime_value
Czech Republic,2,273.24,9.11,136.62
United Kingdom,3,245.52,8.77,81.84
India,2,183.15,8.72,91.58
Germany,4,334.62,8.16,83.66
USA,13,1040.49,7.94,80.04
France,5,389.07,7.78,77.81
Canada,8,535.59,7.05,66.95
Brazil,5,427.68,7.01,85.54
Portugal,2,185.13,6.38,92.57
Other,15,1094.94,7.45,73.0


In [6]:
%%sql

--average order for all countries

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       i.*
     FROM invoice i
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    ROUND(AVG(average_order), 2)
FROM
    (
    SELECT
        SUM(total) / count(distinct invoice_id) average_order
    FROM country_or_other
    GROUP BY country
    ORDER BY average_order DESC
    );

 * sqlite:///chinook.db
Done.


"ROUND(AVG(average_order), 2)"
7.84


**Sales Data by Country Conclusion**
* The average order in all markets is \$7.84
* One strategy to increase the bottom line would be to target markets with an average sale price that matches this and try to increase the number of customers or increase the average sale per customer. In this case, look at Germany, the United States, and France. The Other market (countries with only 1 customer) matches this criteria also, but the number of languages presents a barrier to efficient marketing.
* A corresponding strategy would be to target either the countries with the highest average sale (Czech Republic and the United Kingdom) or the lowest average sale (Brazil and Portugal).
* Lastly, Chinook Music Group could simply target countries with the largest number of existing customers.

### Album vs Track Purchases

In [7]:
%%sql

--determine percent of total sales comprised by album vs individual track

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    ROUND(CAST(count(invoice_id) AS FLOAT) * 100 / (SELECT COUNT(*) FROM invoice), 2) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
no,500,81.43
yes,114,18.57


**Album vs Track Purchases Conclusion**
* Removing customers ability to purchase complete albums would impact about 19% of sales.
* Only some tracks from this group would remain in the library, and the revenue brought from the other tracks would be gone.
* Also, some customers could be buying an albums worth of tracks as a convenience when only looking for a particular few songs. In that case, they could end up purchasing only a portion of the albums tracks that remain in the library.

### Conclusions

* The US market for Chinook Music Group is more than 50% comprised of tracks in the Rock genre. The second most popular genre is Alternative & Punk.
* The three sales support agents are separated by about 3% of Chinook Music Groups total sales for 1st, 2nd, and 3rd place. This analysis could not determine any clear cause other than number of customers.
* The average order in all markets is &7.84. The average sale in the US is only $0.10 more and this is where there are the most customers. Setting a target to increase sale price or number of customers in the US would be good for the bottom line.
* Album sales are responsible for almost 20\% of total volume. Removing them from the library could decrease total revenue.